In [3]:
# import os
# path = r'C:\Users\safre\Documents\Sber_NLP\tn.json'
# print(os.path.basename(path))

# path = 'C:\Users\safre\Documents\Sber_NLP\tn.json'
# path = r''+'C:\Users\safre\Documents\Sber_NLP\tn.json'
# path
# # with open(path, 'r', encoding="utf8") as f:
# #     data_tp = json.loads(f.read())

tn.json


In [26]:
# print('Введите путь к данным')
# path = input()

Введите путь к данным
tp.json


In [16]:
# path = u'test_data.json'
# with open(path, 'r', encoding="utf8") as f:
#     data_test = json.loads(f.read())

# data_test = pd.DataFrame(data_test)

In [1]:
import json
import pandas as pd
import numpy as np
import nltk
# nltk.download("stopwords")
from nltk.corpus import stopwords
import string
import re
from pymystem3 import Mystem
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split

In [2]:
path = u'tn.json'
with open(path, 'r', encoding="utf8") as f:
    data_tn = json.loads(f.read())

with open(path, 'r', encoding="utf8") as f:
    data_tp = json.loads(f.read())

In [3]:
df_tp = pd.DataFrame(data_tp, columns=['sentences'])
df_tp['event'] = np.ones(len(data_tp)).astype(int)

In [4]:
df_tn = pd.DataFrame(data_tn, columns=['sentences'])
df_tn['event'] = np.zeros(len(data_tn)).astype(int)

In [5]:
train_df = pd.concat([df_tn, df_tp], ignore_index = True)

In [6]:
stopwords_ru = stopwords.words("russian")

In [7]:
# def remove_punct(text):    
# string.punctuation
m = Mystem()
train_df.sentences = train_df.sentences.apply(lambda x: m.lemmatize(x))
# train_df.sentences = train_df.sentences.apply(lambda x: x.lower())
train_df

Installing mystem to C:\Users\safre/.local/bin\mystem.exe from http://download.cdn.yandex.net/mystem/mystem-3.1-win-64bit.zip


,sentences,event
0,"[причем, , часть, , котельный, , быть, , з...",0
1,"[ооо, «, балтинвестстрой, » , взять, , обяз...",0
2,"[школа, , на, , южный, , шоссе, , на, , 8...",0
3,"[школа, , на, , южный, , шоссе, , планиров...",0
4,"[беглов, , перечислять, , школа, , , который...",0
...,...,...
2675,"[подрядчик, , ооо, «, альянсдорстрой, » , до...",1
2676,"[он, , заявлять, , , что, , запуск, , ракет...",1
2677,"[застройщик, , ЖК, "", медовый, , долина, "" ...",1
2678,"[официальный, , срок, , , указывать, , в, ,...",1


In [7]:
vectorizer = CountVectorizer(max_features=1500, min_df=5, max_df=0.7, stop_words=stopwords_ru)
train_df_trans = vectorizer.fit_transform(train_df.sentences)

In [11]:
vectorizer.get_feature_names()

['000',
 '03',
 '04',
 '06',
 '09',
 '10',
 '100',
 '11',
 '12',
 '13',
 '14',
 '15',
 '16',
 '17',
 '170',
 '18',
 '19',
 '20',
 '2003',
 '2007',
 '2009',
 '2010',
 '2011',
 '2012',
 '2013',
 '2014',
 '2015',
 '2016',
 '2017',
 '2018',
 '2019',
 '2020',
 '2021',
 '21',
 '214',
 '22',
 '24',
 '25',
 '27',
 '28',
 '29',
 '30',
 '300',
 '32',
 '333',
 '336',
 '37',
 '38',
 '40',
 '42',
 '450',
 '48',
 '50',
 '500',
 '562',
 '58',
 '60',
 '65',
 '70',
 '73',
 '74',
 '740',
 '80',
 '96',
 'com',
 'ru',
 'ura',
 'znak',
 'аварии',
 'аварийного',
 'аварийным',
 'аварийных',
 'августа',
 'августе',
 'агентства',
 'агентство',
 'агрономическая',
 'администрации',
 'администрацию',
 'администрация',
 'адресу',
 'активно',
 'акты',
 'александр',
 'алексей',
 'андерсен',
 'андрей',
 'ао',
 'апартаментов',
 'апреле',
 'апрель',
 'апреля',
 'арбитражном',
 'арбитражный',
 'аренду',
 'аренды',
 'артстройтехно',
 'архитектуры',
 'аэропорта',
 'аэропорту',
 'балтийская',
 'банкротства',
 'беглов',
 'б

In [32]:
X_train, X_test, y_train, y_test = train_test_split(train_df.sentences, train_df.event, test_size=0.3, random_state = 17)